In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
df_sample = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
X_train = df_train.drop(['id', 'target'], axis = 1)
X_test = df_test.drop(['id'], axis = 1)
y_train = df_train['target']

r_seed = 2021

In [ ]:
#params tuning by optuna
if False:
    import optuna.integration.lightgbm as lgb

    params = {'objective': 'binary',
              'metric' : 'auc',
              'seed' : r_seed}

    X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.8, random_state=r_seed)

    train_data = lgb.Dataset(X_tr, label=y_tr)
    valid_data = lgb.Dataset(X_va, label=y_va, reference = train_data)


    model = lgb.train({**params},
                      train_data, 
                      10000, 
                      valid_sets = [train_data, valid_data],
                      early_stopping_rounds=100,#1000, 
                      verbose_eval=500,
                     )

    best_params = model.params
    best_params

In [ ]:
import lightgbm as lgb

best_params = {'objective': 'binary',
 'metric': 'auc',
 'seed': r_seed,
 'feature_pre_filter': False,
 'lambda_l1': 6.156915844651579,
 'lambda_l2': 2.666780092846007e-07,
 'num_leaves': 3,
 'feature_fraction': 0.41600000000000004,
 'bagging_fraction': 0.7763372575672345,
 'bagging_freq': 4,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 100}


y_preds = []
oof = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=r_seed)
for _fold, (train_idx, valid_idx) in enumerate(cv.split(X_train)):
    train_data = lgb.Dataset(X_train.loc[train_idx, :], label=y_train[train_idx])
    valid_data = lgb.Dataset(X_train.loc[valid_idx, :], label=y_train[valid_idx], reference = train_data)

    model = lgb.train({**best_params},
                      train_data, 
                      10000, 
                      valid_sets = [train_data, valid_data],
                      early_stopping_rounds=100,#1000, 
                      verbose_eval=100)
    
    oof[valid_idx] = model.predict(X_train.loc[valid_idx, :], num_iteration = model.best_iteration)
    y_pred = model.predict(X_test, num_iteration= model.best_iteration)
    
    y_preds.append(y_pred)


In [ ]:
df_sample['target'] = np.mean(y_preds, axis = 0)
df_sample.to_csv('sub.csv', index = False)
df_sample.head()